In [181]:
from lxml import etree, html
from lxml.html.clean import clean_html
import pandas as pd
from datetime import date, datetime
import requests
import re
import os
import shutil
from selenium import webdriver

In [183]:
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

def pdf2txt(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text

# Mueve todos los fichero del directorio de trabajo a otro de archivo

In [185]:
source_dir = './DOCMs'
target_dir = './DOCMs_Anteriores'
    
file_names = os.listdir(source_dir)
    
for file_name in file_names:
    #shutil.move(os.path.join(source_dir, file_name), target_dir)
    # Evita que de error si el fichero que se mueve ya existe en dir destino
    try:
        os.remove(os.path.join(target_dir, file_name))
        shutil.move(os.path.join(source_dir, file_name), target_dir)
    except OSError:
        shutil.move(os.path.join(source_dir, file_name), target_dir)

# Recoge resumen diario del DOCM de hoy

In [187]:

today = date.today()

# dd/mm/YYYY
hoy = today.strftime("%Y%m%d")
print("Fecha de Hoy =", hoy)

# dd
d = today.strftime("%d")
print("dia =", d)

# mm
m = today.strftime("%m")
print("mes =", m)

# YYYY
Y = today.strftime("%Y")
print("año =", Y)

print(today.strftime("%d/%m/%Y"))

Fecha de Hoy = 20210308
dia = 08
mes = 03
año = 2021
08/03/2021


In [189]:
URL_HTML_resumen =  "https://docm.jccm.es/portaldocm/cambiarBoletin.do?fecha=" + str(hoy)

In [191]:
 # URL_HTML_resumen =  "https://docm.jccm.es/portaldocm/cambiarBoletin.do?fecha=20210305"

In [193]:
URL_HTML_resumen

'https://docm.jccm.es/portaldocm/cambiarBoletin.do?fecha=20210308'

In [195]:
# carga página HTML y genera árbol

response = requests.get(URL_HTML_resumen)
sumario_HTML = html.fromstring(response.text)

## Recoge Nombre Secciones Sumario

secciones = sumario_HTML.xpath('//*[@class="cabeceraCategoria"]/text()')
secciones = [ re.sub('\r|\n|\t','', seccion) for seccion in secciones ]

In [197]:
secciones

['II.- AUTORIDADES Y PERSONAL',
 'III.- OTRAS DISPOSICIONES Y ACTOS',
 'IV.- ADMINISTRACIÓN DE JUSTICIA',
 'V.- ANUNCIOS']

In [199]:
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)

save_html(response.content, './DOCMs/Resumen-DOCM-' + hoy + '.html')

In [201]:
for seccion in sumario_HTML.xpath('//*[@class="cabeceraCategoria"]'):
    nombre_seccion = seccion.xpath('./text()')
    nombre_seccion = str(nombre_seccion[0]).strip()
    print(nombre_seccion)


II.- AUTORIDADES Y PERSONAL
III.- OTRAS DISPOSICIONES Y ACTOS
IV.- ADMINISTRACIÓN DE JUSTICIA
V.- ANUNCIOS


In [203]:
DOCM_sumarios = pd.DataFrame(columns=['seccion','item_Title','item_urlHTML','item_urlPDF'])

lista = []
categoria = []
for sumario in sumario_HTML.xpath('//*[@class="sumario"]'):
    lista.append(sumario.text_content().strip())
    print(sumario.xpath('./preceding::h3[1]/text()'))
    DOCM_sumarios['seccion'] = sumario.xpath('./preceding::h3/text()')

DOCM_sumarios['item_Title'] = lista
#DOCM_sumarios['seccion'] = categoria

lista = []   
for link_HTML in sumario_HTML.xpath('//*[@title="Ver los datos detallados del documento"]'):
    link = link_HTML.xpath('./@href')
    #print(link)
    lista.append('https://docm.jccm.es/portaldocm' + str(link)[3:-2])

DOCM_sumarios['item_urlHTML'] = lista

lista = []   
for link_PDF in sumario_HTML.xpath('//div/a[@class="new-window"]'):
    link = link_PDF.xpath('./@href')
    #print(link)
    lista.append('https://docm.jccm.es/portaldocm' + str(link)[3:-2])

DOCM_sumarios['item_urlPDF'] = lista


In [205]:
# Compone la primera tabla con campos acceso a detalle disposiciones individuales 

for sumario in sumario_HTML.xpath('//*[@class="sumario"]'):
    title = sumario.text_content().strip()
    print(sumario)
    categoria = sumario.xpath('./preceding::h3[1]/text()')
    print(categoria)
    link_HTML = sumario.xpath('following-sibling::div[1]/a[@title="Ver los datos detallados del documento"]/@href')
    link_HTML = 'https://docm.jccm.es/portaldocm' + str(link_HTML)[3:-2]
    print(link_HTML)
    link_PDF = sumario.xpath('./a/@href')
    link_PDF = 'https://docm.jccm.es/portaldocm' + str(link_PDF)[3:-2]
    print(link_PDF)

    DOCM_sumarios = DOCM_sumarios.append({'seccion': categoria, 
                                          'item_Title': title, 
                                          'item_urlHTML' : link_HTML,
                                          'item_urlPDF': link_PDF}, 
                                          ignore_index = True)


<Element p at 0x7f98d6cdae50>
['\r\n\t\t\t\t\tII.- AUTORIDADES Y PERSONAL\r\n\t\t\t']
https://docm.jccm.es/portaldocm/detalleDocumento.do?idDisposicion=1614592694266560102
https://docm.jccm.es/portaldocm/descargarArchivo.do?ruta=2021/03/08/pdf/2021_2336.pdf&tipo=rutaDocm
<Element p at 0x7f98da26e360>
['\r\n\t\t\t\t\tII.- AUTORIDADES Y PERSONAL\r\n\t\t\t']
https://docm.jccm.es/portaldocm/detalleDocumento.do?idDisposicion=1614766129129540718
https://docm.jccm.es/portaldocm/descargarArchivo.do?ruta=2021/03/08/pdf/2021_2446.pdf&tipo=rutaDocm
<Element p at 0x7f98da425cc0>
['\r\n\t\t\t\t\tII.- AUTORIDADES Y PERSONAL\r\n\t\t\t']
https://docm.jccm.es/portaldocm/detalleDocumento.do?idDisposicion=1614605957943560334
https://docm.jccm.es/portaldocm/descargarArchivo.do?ruta=2021/03/08/pdf/2021_2371.pdf&tipo=rutaDocm
<Element p at 0x7f98da425d60>
['\r\n\t\t\t\t\tIII.- OTRAS DISPOSICIONES Y ACTOS\r\n\t\t\t']
https://docm.jccm.es/portaldocm/detalleDocumento.do?idDisposicion=1614616566800540398
https:

lista = []
for sumario in sumario_HTML.xpath('//*[@class="sumario"]'):
    lista.append(sumario.text_content().strip())

DOCM_sumarios['item_Title'] = lista

lista = []   
for link_HTML in sumario_HTML.xpath('//*[@title="Ver los datos detallados del documento"]'):
    link = link_HTML.xpath('./@href')
    #print(link)
    lista.append('https://docm.jccm.es/portaldocm' + str(link)[3:-2])

DOCM_sumarios['item_urlHTML'] = lista

lista = []   
for link_PDF in sumario_HTML.xpath('//div/a[@class="new-window"]'):
    link = link_PDF.xpath('./@href')
    #print(link)
    lista.append('https://docm.jccm.es/portaldocm' + str(link)[3:-2])

DOCM_sumarios['item_urlPDF'] = lista

In [207]:
# Convierte columna seccion a string
DOCM_sumarios['seccion'] = DOCM_sumarios['seccion'].astype(str)

In [209]:
# Elimina caracteres \n \t y \r
DOCM_sumarios.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)

In [211]:
# Elimina disposiciones con seccion que contenga las palabras AUTORIDADES y/o PERSONAL
terminos = ['AUTORIDADES Y PERSONAL', 'AUTORIDADES', 'PERSONAL']
DOCM_sumarios = DOCM_sumarios.drop(DOCM_sumarios.index[DOCM_sumarios['seccion'].str.contains('|'.join(terminos), na=False)])

In [213]:
# Elimina disposiciones con item_Title que contenga las palabras Sanciones
terminos = ['Sanciones']
DOCM_sumarios = DOCM_sumarios.drop(DOCM_sumarios.index[DOCM_sumarios['item_Title'].str.contains('|'.join(terminos), na=False)])

In [215]:
len(DOCM_sumarios)

63

In [217]:
DOCM_sumarios

,seccion,item_Title,item_urlHTML,item_urlPDF
3,['III.- OTRAS DISPOSICIONES Y ACTOS'],"Gabinete Jurídico. Resolución de 01/03/2021, d...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...
4,['III.- OTRAS DISPOSICIONES Y ACTOS'],"Notificaciones. Notificación de 22/02/2021, de...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...
5,['III.- OTRAS DISPOSICIONES Y ACTOS'],Ayudas y Subvenciones. Resolución de 02/03/202...,https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...
8,['III.- OTRAS DISPOSICIONES Y ACTOS'],Administración Electrónica. Resolución de 25/0...,https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...
9,['III.- OTRAS DISPOSICIONES Y ACTOS'],"Espectáculos. Resolución de 01/03/2021, de la ...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...
...,...,...,...,...
95,['V.- ANUNCIOS'],"Anuncio de 25/02/2021, del Ayuntamiento de Par...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...
96,['V.- ANUNCIOS'],"Anuncio de 19/02/2021, del Ayuntamiento de Tam...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...
97,['V.- ANUNCIOS'],"Anuncio de 19/02/2021, del Ayuntamiento de Tam...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...
98,['V.- ANUNCIOS'],"Anuncio de 23/02/2021, del Ayuntamiento de Tem...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...


In [227]:
print(DOCM_sumarios['item_urlHTML'])

3     https://docm.jccm.es/portaldocm/detalleDocumen...
4     https://docm.jccm.es/portaldocm/detalleDocumen...
5     https://docm.jccm.es/portaldocm/detalleDocumen...
8     https://docm.jccm.es/portaldocm/detalleDocumen...
9     https://docm.jccm.es/portaldocm/detalleDocumen...
                            ...                        
95    https://docm.jccm.es/portaldocm/detalleDocumen...
96    https://docm.jccm.es/portaldocm/detalleDocumen...
97    https://docm.jccm.es/portaldocm/detalleDocumen...
98    https://docm.jccm.es/portaldocm/detalleDocumen...
99    https://docm.jccm.es/portaldocm/detalleDocumen...
Name: item_urlHTML, Length: 63, dtype: object


In [221]:
# response = requests.get(DOCM_sumarios['item_urlHTML'][0])
# 
# sumario_HTML = html.fromstring(response.text)
# print(response.text)

In [223]:
DOCM_sumarios.dtypes

seccion         object
item_Title      object
item_urlHTML    object
item_urlPDF     object
dtype: object

### Necesita libreria Selenium para renderizar JS script

options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)

driver.get(DOCM_sumarios['item_urlHTML'][0])
#print(driver.page_source)
response = driver.page_source
sumario_HTML = html.fromstring(response)

numero_diario = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[1]/td[2]/text()')
numero_pagina = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[3]/td[4]/text()')
NID = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[2]/td[2]/text()')
rango = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[5]/td[2]/text()')
organo_emisor = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[7]/td[2]/text()')

print(numero_diario[0].strip(), numero_pagina[0].strip(), NID[0].strip(), rango[0].strip(), organo_emisor[0].strip())
print()

driver.quit()

In [228]:
### Recoge información de página de detalle con Selenium
### es necesario ya que la página se genera con un JS y
### hay que renderizarla con un headless web browser

options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)

DOCM_sumarios['NID'] = ''
DOCM_sumarios['numero_diario'] = ''
DOCM_sumarios['numero_pagina'] = ''
DOCM_sumarios['rango'] = ''
DOCM_sumarios['organo_emisor'] = ''
DOCM_sumarios['Fecha_publicacion'] = ''

for i, row in DOCM_sumarios.iterrows():
    # carga página HTML y genera árbol
    driver.get(row['item_urlHTML'])
    #print(driver.page_source)
    response = driver.page_source
    
    sumario_HTML = html.fromstring(response)
    numero_diario = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[1]/td[2]/text()')
    numero_pagina = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[3]/td[4]/text()')
    NID = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[2]/td[2]/text()')
    rango = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[5]/td[2]/text()')
    organo_emisor = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[7]/td[2]/text()')
    item_itle = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[9]/td[2]/text()')

    #print(numero_diario[0].strip(), numero_pagina[0].strip(), NID[0].strip(), rango[0].strip(), organo_emisor[0].strip())

    DOCM_sumarios['NID'][i] = NID[0].strip()
    DOCM_sumarios['numero_diario'][i] = numero_diario[0].strip()
    DOCM_sumarios['numero_pagina'][i] = numero_pagina[0].strip()
    DOCM_sumarios['rango'][i] = rango[0].strip()
    DOCM_sumarios['organo_emisor'][i] = organo_emisor[0].strip()

    DOCM_sumarios['Fecha_publicacion'][i] = datetime.strptime(str(today),"%Y-%m-%d").date()

driver.quit()

In [229]:
DOCM_sumarios

,seccion,item_Title,item_urlHTML,item_urlPDF,NID,numero_diario,numero_pagina,rango,organo_emisor,Fecha_publicacion
3,['III.- OTRAS DISPOSICIONES Y ACTOS'],"Gabinete Jurídico. Resolución de 01/03/2021, d...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/2370,45/2021,9563,Resolución,Vicepresidencia,2021-03-08
4,['III.- OTRAS DISPOSICIONES Y ACTOS'],"Notificaciones. Notificación de 22/02/2021, de...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/2359,45/2021,9566,Notificación,Secretaría General,2021-03-08
5,['III.- OTRAS DISPOSICIONES Y ACTOS'],Ayudas y Subvenciones. Resolución de 02/03/202...,https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/2523,45/2021,9567,Resolución,Dirección General de Empresas,2021-03-08
8,['III.- OTRAS DISPOSICIONES Y ACTOS'],Administración Electrónica. Resolución de 25/0...,https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/2348,45/2021,9636,Resolución,Viceconsejería de Administración Local y Coord...,2021-03-08
9,['III.- OTRAS DISPOSICIONES Y ACTOS'],"Espectáculos. Resolución de 01/03/2021, de la ...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/2302,45/2021,9637,Resolución,Delegación Provincial de Hacienda y Administra...,2021-03-08
...,...,...,...,...,...,...,...,...,...,...
95,['V.- ANUNCIOS'],"Anuncio de 25/02/2021, del Ayuntamiento de Par...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/2141,45/2021,9741,Anuncio,Ayuntamiento de Parrillas (Toledo),2021-03-08
96,['V.- ANUNCIOS'],"Anuncio de 19/02/2021, del Ayuntamiento de Tam...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/1834,45/2021,9742,Anuncio,Ayuntamiento de Tamajón (Guadalajara),2021-03-08
97,['V.- ANUNCIOS'],"Anuncio de 19/02/2021, del Ayuntamiento de Tam...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/1835,45/2021,9743,Anuncio,Ayuntamiento de Tamajón (Guadalajara),2021-03-08
98,['V.- ANUNCIOS'],"Anuncio de 23/02/2021, del Ayuntamiento de Tem...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/1980,45/2021,9744,Anuncio,Ayuntamiento de Tembleque (Toledo),2021-03-08


# Salva PDFs y Genera DF con datos Análisis de cada PDF

In [230]:
# Crea nueva columna vacía de tipo lista en tabla_analisis
#DOCM_sumarios['Referencias_completas'] = [[] for i in range(len(tabla_analisis))]
DOCM_sumarios['Tags'] = [[] for i in range(len(DOCM_sumarios))]
DOCM_sumarios['Match_ASECORP_BBDD'] = [[] for i in range(len(DOCM_sumarios))]


In [231]:
pattern = ['Ley [0-9]+\/[0-9]+','Ley Orgánica [0-9]+\/[0-9]+','Decreto [0-9]+\/[0-9]+','Real Decreto [0-9]+\/[0-9]+','Real Decreto Legislativo [0-9]+\/[0-9]+','Real Decreto-ley [0-9]+\/[0-9]+','Orden [A-Z]+\/[0-9]+\/[0-9]+','Orden Circular [0-9]+\/[0-9]+','Reglamento \(UE\) [0-9]+\/[0-9]+', 'Reglamento de Ejeución \(UE\) [0-9]+\/[0-9]+' ,'Sentencia de [0-9]+ de [a-z]+ de [0-9]+','Sentencia [0-9]+\/[0-9]+','Orden de [0-9]+ de [a-z]+ de [0-9]+', 'Resolución de [0-9]+ de [a-z]+ de [0-9]+','Resolución [a-z]+\/[0-9]+\/[0-9]+', 'Nota de Servicio [0-9]+\/[0-9]+', 'Acuerdo multilateral M\-[0-9]+', 'Acuerdo Multilateral RID [0-9]+\/[0-9]+', 'Circular [0-9]+\/[0-9]+', 'Decisión \(UE\) [0-9]+\/[0-9]+', 'Decisión de Ejecución \(UE\) [0-9]+\/[0-9]+', 'Instrucción IS\-[0-9]+']

In [232]:
for i, row in DOCM_sumarios.iterrows():
    r = requests.get(row['item_urlPDF'])

    # Salva PDFs de enlaces a items
    filename = row['NID'].replace('/','-' )
    f = './DOCMs/' + 'DOCM_NID_' + filename + '.pdf'
    save_html(r.content, f)

    # Extrae texto de PDFs
    pdf_contents = pdf2txt(f)
    #print(pdf_contents)

    # Busca expresiones REGX coincidentes con Patrones definidos
    DOCM_sumarios['Tags'][i] = re.findall('|'.join(pattern), str(pdf_contents), flags=re.IGNORECASE)
    print(DOCM_sumarios['Tags'][i])

['Decreto 77/2019', 'Ley 5/2013', 'Ley 5/2013']
['Ley 39/2015', 'Ley 39/2015', 'Ley 39/2015', 'Ley 29/1998']
['Ley 38/2003', 'RESOLUCIÓN DE 2 DE MARZO DE 2021', 'Decreto 21/2008', 'Ley 39/2015', 'Decreto 12/2010', 'Ley 38/2003', 'Real Decreto 887/2006', 'Orden HFP/1979/2016', 'Decreto 21/2008', 'Ley 38/2003', 'Ley 11/2003', 'Ley 22/2003', 'Ley 31/1995', 'Decreto 62/2020', 'Decreto 9/2020', 'Decreto 21/2008', 'Ley 5/2020', 'Decreto 31/2017', 'Ley 5/2017', 'Decreto 52/2018', 'Ley 39/2015', 'Ley 38/2003', 'Ley 9/2017', 'Real Decreto 1414/2006', 'Real Decreto Legislativo 1/2013', 'Ley 39/2015', 'Ley 39/2015', 'Decreto 21/2008', 'Ley 39/2015', 'Decreto 21/2008', 'Decreto 21/2008', 'Ley 39/2015', 'Decreto 21/2008', 'Ley 39/2015', 'Real Decreto 887/2006', 'Ley 38/2003', 'Ley 39/2015', 'Ley 38/2003', 'Ley 38/2003', 'Ley 11/2003', 'Ley 31/1995', 'Ley Orgánica 3/2018', 'Ley 38/2003', 'Ley 39/2015', 'Decreto 21/2008', 'Decreto 21/2008', 'Decreto 21/2008', 'Ley 58/2003', 'Real Decreto 209/2003', '

In [233]:
# Elimina Tags duplicados
for i, row in DOCM_sumarios.iterrows():
    DOCM_sumarios['Tags'][i] = list(set(DOCM_sumarios['Tags'][i]))
    print(DOCM_sumarios['Tags'][i])

['Ley 5/2013', 'Decreto 77/2019']
['Ley 29/1998', 'Ley 39/2015']
['Ley 9/1999', 'Decreto 21/2008', 'Ley 21/2013', 'Ley Orgánica 3/2018', 'Ley 5/2020', 'Ley 38/2003', 'Ley 22/2003', 'Decreto 9/2020', 'Ley 5/2017', 'Ley 4/2016', 'Ley 9/2017', 'Orden HFP/1979/2016', 'Ley 11/2003', 'Decreto 12/2010', 'Real Decreto 887/2006', 'Ley 31/1995', 'Decreto 62/2020', 'Ley 42/2007', 'Real Decreto Legislativo 1/2013', 'Decreto 52/2018', 'Ley 9/2012', 'Decreto 31/2017', 'Real Decreto 1414/2006', 'Ley 39/2015', 'RESOLUCIÓN DE 2 DE MARZO DE 2021', 'Ley 58/2003', 'Real Decreto 209/2003']
['Resolución de 27 de julio de 2017', 'Resolución de 3 de mayo de 2017', 'Decreto 82/2015']
['Decreto 73/2014', 'Decreto 38/2013', 'Decreto 60/2016', 'Real Decreto 145/1996']
['ley 39/2015']
['Ley 39/2015']
['Ley 39/2015']
['Ley 39/2015']
['Ley 29/1998']
['Ley 39/2015']
['Ley 36/2011', 'Ley 39/2015']
['Ley 36/2011', 'Ley 39/2015']
['Ley 39/2015', 'Decreto 80/2012', 'Decreto 108/2014']
['Ley 39/2015', 'Decreto 80/2012', '

In [234]:
# Aplica expresiones REGEX para búsqueda de leyes, decretos, etc. referenciadas anteriormente
regex_result = []
[ regex_result.append(tag) for tags in DOCM_sumarios['Tags'] for tag in tags ]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [235]:
print(regex_result)

['Ley 5/2013', 'Decreto 77/2019', 'Ley 29/1998', 'Ley 39/2015', 'Ley 9/1999', 'Decreto 21/2008', 'Ley 21/2013', 'Ley Orgánica 3/2018', 'Ley 5/2020', 'Ley 38/2003', 'Ley 22/2003', 'Decreto 9/2020', 'Ley 5/2017', 'Ley 4/2016', 'Ley 9/2017', 'Orden HFP/1979/2016', 'Ley 11/2003', 'Decreto 12/2010', 'Real Decreto 887/2006', 'Ley 31/1995', 'Decreto 62/2020', 'Ley 42/2007', 'Real Decreto Legislativo 1/2013', 'Decreto 52/2018', 'Ley 9/2012', 'Decreto 31/2017', 'Real Decreto 1414/2006', 'Ley 39/2015', 'RESOLUCIÓN DE 2 DE MARZO DE 2021', 'Ley 58/2003', 'Real Decreto 209/2003', 'Resolución de 27 de julio de 2017', 'Resolución de 3 de mayo de 2017', 'Decreto 82/2015', 'Decreto 73/2014', 'Decreto 38/2013', 'Decreto 60/2016', 'Real Decreto 145/1996', 'ley 39/2015', 'Ley 39/2015', 'Ley 39/2015', 'Ley 39/2015', 'Ley 29/1998', 'Ley 39/2015', 'Ley 36/2011', 'Ley 39/2015', 'Ley 36/2011', 'Ley 39/2015', 'Ley 39/2015', 'Decreto 80/2012', 'Decreto 108/2014', 'Ley 39/2015', 'Decreto 80/2012', 'Decreto 108/20

In [236]:
## Elimina duplicados
boletin_flat_list = list(set(regex_result))

In [237]:
boletin_flat_list

['Ley 29/1998',
 'Real Decreto 328/2003',
 'Ley 34/2007',
 'Ley 38/2003',
 'Real Decreto 1084/2005',
 'Real Decreto 887/2006',
 'Decreto 78/2016',
 'Ley 31/1995',
 'Decreto 62/2020',
 'Decreto 87/2019',
 'Ley 39/15',
 'Ley 9/2012',
 'Decreto 31/2017',
 'Ley 9/2018',
 'Decreto 80/2012',
 'Real Decreto 209/2003',
 'Ley 26/2015',
 'Ley 2/98',
 'Real Decreto 100/2011',
 'Decreto 20/2004',
 'Real Decreto 1432/2008',
 'Decreto 21/2008',
 'Decreto 73/2014',
 'Ley Orgánica 3/2018',
 'Decreto 77/2019',
 'Real Decreto 833/1998',
 'Ley 5/2020',
 'Decreto 178/2010',
 'Ley 22/2003',
 'Decreto 9/2020',
 'Ley 4/2013',
 'Ley 11/2003',
 'Real Decreto 105/2008',
 'Real Decreto 692/2010',
 'Decreto 52/2018',
 'Real Decreto 1414/2006',
 'Decreto 34/2017',
 'Ley 5/2013',
 'Decreto 86/2018',
 'Decreto 80/2007',
 'Orden APM/1007/2017',
 'Ley 9/1999',
 'Resolución de 12 de febrero de 2021',
 'Ley 1/2009',
 'Real Decreto 145/1996',
 'Ley 36/2011',
 'Decreto 60/2016',
 'Real Decreto 952/1997',
 'Decreto 82/2015

## Importa BBDD ASECORP

In [238]:
ASECORP_BBDD = pd.read_csv('./ASECORP/ExportNormes_20210126.csv', delimiter=';')

In [239]:
ASECORP_BBDD

,Codigo,Numero,Titulo,CodigoAmbito,Ambito,Municipio,TiposNorma,IdTiposNorma
0,RES SLT/2313/2020 CAT,2313,"Resolución SLT/2313/2020, de 24 de septiembre,...",CAT,Cataluña,NaN,Prevención,[2]
1,(zzz) RES 12/4/2007 AST,12/4/2007,"Resolución de 12 abril de 2007, de la Consejer...",AST,Asturias,NaN,M. Ambiente,[1]
2,A CORUÑA 10/7/2015,10/7/2015,Modificación de la Ordenanza de vertidos y ser...,GAL,Galicia,A Coruña,M. Ambiente,[1]
3,A CORUÑA 11/8/2014 (1),11/8/2014,Ordenanza municipal reguladora de los procedim...,GAL,Galicia,A Coruña,M. Ambiente,[1]
4,A CORUÑA 11/8/2014 (2),11/8/2014,Ordenanza de protección contra la contaminació...,GAL,Galicia,A Coruña,M. Ambiente,[1]
...,...,...,...,...,...,...,...,...
19152,ZUMAIA 1/3/2001,1/3/2001,Ordenanza reguladora de la gestión del ciclo i...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]
19153,ZUMAIA 12/11/1999,12/11/1999,Normativa complementaria de las Normas Subsidi...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]
19154,ZUMAIA 15/6/2012,15/6/2012,Modificación de la Ordenanza municipal de la g...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]
19155,ZUMAIA 6/2/2004,6/2/2004,Modificación de la Normativa complementaria de...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]


In [240]:
ASECORP_BBDD_BOE = ASECORP_BBDD.loc[ASECORP_BBDD['Ambito'] == 'España']

In [241]:
ASECORP_BBDD_BOE

,Codigo,Numero,Titulo,CodigoAmbito,Ambito,Municipio,TiposNorma,IdTiposNorma
23,ACU 1/2020 ESP,1,Acuerdo Multilateral RID 1/2020 en virtud de l...,ESP,España,NaN,M. Ambiente / Prevención,[1][2]
35,ACU 11/6/2007 ESP,11/6/2007,Entrada en vigor del Acuerdo entre el Reino de...,ESP,España,NaN,M. Ambiente,[1]
57,ACU 17/10/2006 ESP,17/10/2006,Aplicación provisional del Acuerdo entre el Re...,ESP,España,NaN,M. Ambiente,[1]
71,ACU 19/11/2018 ESP,19/11/2018,Acuerdo de la Junta de Gobierno de la Confeder...,ESP,España,NaN,M. Ambiente,[1]
82,ACU 2/2020 ESP,2,Acuerdo Multilateral RID 2/2020 en virtud de l...,ESP,España,NaN,M. Ambiente / Prevención,[1][2]
...,...,...,...,...,...,...,...,...
18496,STC 86/2019 ESP,86,"Sentencia 86/2019, de 20 de junio de 2019. Rec...",ESP,España,NaN,M. Ambiente,[1]
18497,STC 87/2019 ESP,87,"Sentencia 87/2019, de 20 de junio de 2019. Rec...",ESP,España,NaN,M. Ambiente,[1]
18498,STC 88/2018 ESP,88,"Sentencia 88/2018, de 19 de julio de 2018. Con...",ESP,España,NaN,M. Ambiente / Reg. Industrial,[1][3]
18499,STC 9/3/2010 ESP,9/3/2010,"Sentencia de 9 de marzo de 2010, de la Sala Te...",ESP,España,NaN,M. Ambiente,[1]


In [242]:
# Crea nueva columna vacía de tipo lista en ASECORP_BBDD
ASECORP_BBDD_BOE['Tags'] = [[] for i in range(len(ASECORP_BBDD_BOE))]

In [243]:
titulo = []
for i, row in ASECORP_BBDD_BOE.iterrows():
    titulo.append(str(row['Titulo']) + str(re.findall('|'.join(pattern), str(row['Titulo']), flags=re.IGNORECASE)))
    ASECORP_BBDD_BOE['Tags'][i] = re.findall('|'.join(pattern), str(row['Titulo']), flags=re.IGNORECASE)

In [244]:
ASECORP_BBDD_BOE[1200:1250]

,Codigo,Numero,Titulo,CodigoAmbito,Ambito,Municipio,TiposNorma,IdTiposNorma,Tags
11629,ORD PRE/2827/2009 ESP,2827,"Orden PRE/2827/2009, de 19 de octubre, por la ...",ESP,España,NaN,M. Ambiente,[1],"[Orden PRE/2827/2009, Real Decreto 1370/2006]"
11630,ORD PRE/2843/2009 ESP,2843,"Orden PRE/2843/2009, de 19 de octubre, por la ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2843/2009, Real Decreto 2163/1994]"
11631,ORD PRE/2851/2010 ESP,2851,"Orden PRE/2851/2010, de 4 de noviembre, por la...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2851/2010, Real Decreto 2163/1994]"
11632,ORD PRE/2871/2011 ESP,2871,"Orden PRE/2871/2011, de 25 de octubre, por la ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2871/2011, Real Decreto 2163/1994]"
11633,ORD PRE/2872/2011 ESP,2872,"Orden PRE/2872/2011, de 25 de octubre, por la ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2872/2011, Real Decreto 2163/1994]"
11634,ORD PRE/29/2004 ESP,29,"Orden PRE/29/2004, de 15 de enero, por la que ...",ESP,España,NaN,M. Ambiente,[1],"[Orden PRE/29/2004, Real Decreto 837/2002]"
11635,ORD PRE/2922/2005 ESP,2922,"Orden PRE/2922/2005, de 19 de septiembre, por ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2922/2005, Orden de 8 de marzo de 1..."
11636,ORD PRE/2957/2008 ESP,2957,"Orden PRE/2957/2008, de 10 de octubre, por la ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2957/2008, Real Decreto 2163/1994]"
11637,ORD PRE/3/2006 ESP,3,"Orden PRE/3/2006, de 12 de enero, por la que s...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/3/2006, Real Decreto 255/2003]"
11638,ORD PRE/3159/2004 ESP,3159,"Orden PRE/3159/2004, de 28 de septiembre, por ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/3159/2004, Real Decreto 1406/1989]"


In [245]:
# titulo

In [246]:
len(titulo)

2959

In [247]:
## Busca patrones regex definidos en columna títulos del DF 
boletin = []
for i, row in ASECORP_BBDD_BOE.iterrows():
    boletin.append(str(re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE)))

In [248]:
boletin[0:25]

["['Acuerdo Multilateral RID 1/2020']",
 '[]',
 '[]',
 '[]',
 "['Acuerdo Multilateral RID 2/2020']",
 '[]',
 "['Acuerdo Multilateral RID 1/2013']",
 '[]',
 '[]',
 '[]',
 '[]',
 "['Acuerdo Multilateral RID 3/2007']",
 "['Acuerdo Multilateral RID 4/2009']",
 '[]',
 '[]',
 "['Acuerdo Multilateral M-150']",
 "['Acuerdo Multilateral M-159']",
 "['Acuerdo multilateral M-168']",
 "['Acuerdo Multilateral M-170']",
 "['Acuerdo Multilateral M-171']",
 "['Acuerdo Multilateral M-173']",
 "['Acuerdo Multilateral M-175']",
 '[]',
 "['Acuerdo Multilateral M-178']",
 "['Acuerdo Multilateral M-198']"]

In [249]:
## Para cada fila de la BBDD recoge la expresión REGEX encontrada y si no existe no la incluye 
## en la lista resultante llamada 'boletin'. Además si no detecta expresión la cuenta como vacía
## en variable 'n_vacios', y la añade a la lista 'vacios' para inspeccionar posteriormente 
boletin = []
vacios = []
n_vacios = 0
for i, row in ASECORP_BBDD_BOE.iterrows():
    regex_result = re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE)
    if len(regex_result) != 0:
        boletin.append(re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE))
    else:
        n_vacios += 1
        vacios.append(row['Titulo'])

In [250]:
n_vacios

225

In [251]:
vacios[0:25]

['Entrada en vigor del Acuerdo entre el Reino de España y el Principado de Andorra sobre traslado y gestión de residuos, hecho en Madrid el 17 de octubre de 2006.',
 'Aplicación provisional del Acuerdo entre el Reino de España y el Principado de Andorra sobre traslado y gestión de residuos, hecho en Madrid el 17 de octubre de 2006.',
 'Acuerdo de la Junta de Gobierno de la Confederación Hidrográfica del Segura celebrada el día 30 de octubre de 2018 sobre aplicación de indisponibilidad o reducción de dotaciones de acuerdo al Plan Especial de Sequía y al Real Decreto que la regula',
 'Entrada en vigor del Acuerdo entre el Reino de España y el Principado de Andorra sobre el traslado de residuos, hecho en Madrid el 29 de noviembre de 2011.',
 'Canje de Notas por el que se procede a la prórroga del mantenimiento en vigor del Acuerdo entre el Reino de España y el Principado de Andorra sobre traslado y gestión de residuos, hecho en Madrid el 27 de enero de 2000.',
 'Aplicación provisional del

In [252]:
boletin[0:25]

[['Acuerdo Multilateral RID 1/2020'],
 ['Acuerdo Multilateral RID 2/2020'],
 ['Acuerdo Multilateral RID 1/2013'],
 ['Acuerdo Multilateral RID 3/2007'],
 ['Acuerdo Multilateral RID 4/2009'],
 ['Acuerdo Multilateral M-150'],
 ['Acuerdo Multilateral M-159'],
 ['Acuerdo multilateral M-168'],
 ['Acuerdo Multilateral M-170'],
 ['Acuerdo Multilateral M-171'],
 ['Acuerdo Multilateral M-173'],
 ['Acuerdo Multilateral M-175'],
 ['Acuerdo Multilateral M-178'],
 ['Acuerdo Multilateral M-198'],
 ['Acuerdo Multilateral M-214'],
 ['Acuerdo Multilateral M-218'],
 ['Acuerdo Multilateral M-226'],
 ['Acuerdo Multilateral M-228'],
 ['Acuerdo Multilateral M-231'],
 ['Acuerdo Multilateral M-236'],
 ['Acuerdo Multilateral M-245'],
 ['Acuerdo Multilateral M-254'],
 ['Acuerdo Multilateral M-259'],
 ['Acuerdo Multilateral M-265'],
 ['Acuerdo Multilateral M-268']]

In [253]:
# Flatten list of lists
boletin_ASECORP_flat_list = [item for sublist in boletin for item in sublist]

In [254]:
boletin_ASECORP_flat_list[0:25]

['Acuerdo Multilateral RID 1/2020',
 'Acuerdo Multilateral RID 2/2020',
 'Acuerdo Multilateral RID 1/2013',
 'Acuerdo Multilateral RID 3/2007',
 'Acuerdo Multilateral RID 4/2009',
 'Acuerdo Multilateral M-150',
 'Acuerdo Multilateral M-159',
 'Acuerdo multilateral M-168',
 'Acuerdo Multilateral M-170',
 'Acuerdo Multilateral M-171',
 'Acuerdo Multilateral M-173',
 'Acuerdo Multilateral M-175',
 'Acuerdo Multilateral M-178',
 'Acuerdo Multilateral M-198',
 'Acuerdo Multilateral M-214',
 'Acuerdo Multilateral M-218',
 'Acuerdo Multilateral M-226',
 'Acuerdo Multilateral M-228',
 'Acuerdo Multilateral M-231',
 'Acuerdo Multilateral M-236',
 'Acuerdo Multilateral M-245',
 'Acuerdo Multilateral M-254',
 'Acuerdo Multilateral M-259',
 'Acuerdo Multilateral M-265',
 'Acuerdo Multilateral M-268']

In [255]:
## Elimina duplicados
boletin_ASECORP_flat_list = list(set(boletin_ASECORP_flat_list))

In [256]:
boletin_ASECORP_flat_list[0:25]

['Real decreto 1777/2004',
 'Resolución de 14 de junio de 2013',
 'Real Decreto 569/2020',
 'Orden de 30 de junio de 1993',
 'Orden ITC/3863/2007',
 'Orden PRE/77/2008',
 'Ley 1/2019',
 'LEY 13/2003',
 'Nota de servicio 2/2016',
 'Real Decreto 317/2019',
 'Resolución de 11 de octubre de 2017',
 'Orden FOM/510/2018',
 'Real Decreto Legislativo 1/2010',
 'Real Decreto 777/2012',
 'Real Decreto 99/2003',
 'REAL DECRETO 1123/2001',
 'Sentencia de 30 de enero de 2007',
 'Resolución de 10 de julio de 2014',
 'Real Decreto-ley 29/2020',
 'Sentencia de 5 de julio de 2016',
 'Ley 46/1999',
 'Real Decreto 1101/2011',
 'Orden de 3 de julio de 1987',
 'Ley 34/2007',
 'Orden de 31 de marzo de 1986']

In [257]:
## Busca coincidencias entre lista boletines BOEs explorados y lista boletines de BBDD ASECORP
set(boletin_flat_list) & set(boletin_ASECORP_flat_list)

{'Ley 20/1986',
 'Ley 21/2013',
 'Ley 22/2011',
 'Ley 24/2013',
 'Ley 26/2015',
 'Ley 31/1995',
 'Ley 34/2007',
 'Ley 36/2011',
 'Ley 42/2007',
 'Ley 5/2013',
 'Ley 9/2003',
 'Ley 9/2017',
 'Ley 9/2018',
 'Orden APM/1007/2017',
 'Real Decreto 100/2011',
 'Real Decreto 105/2008',
 'Real Decreto 1432/2008',
 'Real Decreto 1528/2012',
 'Real Decreto 952/1997',
 'Real Decreto Legislativo 1/2013',
 'Resolución de 27 de julio de 2017'}

In [258]:
#DOCM_sumarios['Tags'].isin(ASECORP_BBDD_BOE['Tags'])
for row_to_compare in DOCM_sumarios['Tags']:
    for row_comparing in ASECORP_BBDD_BOE['Tags']:
        if set(row_comparing) & set(row_to_compare):
            print(set(row_comparing) & set(row_to_compare))

{'Ley 5/2013'}
{'Ley 21/2013'}
{'Ley 31/1995'}
{'Ley 42/2007'}
{'Ley 42/2007'}
{'Ley 9/2017'}
{'Ley 21/2013'}
{'Ley 42/2007'}
{'Ley 31/1995'}
{'Real Decreto Legislativo 1/2013'}
{'Ley 31/1995'}
{'Ley 31/1995'}
{'Ley 21/2013'}
{'Ley 21/2013'}
{'Ley 21/2013'}
{'Resolución de 27 de julio de 2017'}
{'Ley 36/2011'}
{'Ley 36/2011'}
{'Ley 26/2015'}
{'Ley 20/1986'}
{'Ley 21/2013'}
{'Ley 22/2011'}
{'Ley 31/1995'}
{'Ley 34/2007'}
{'Ley 22/2011'}
{'Ley 9/2018', 'Ley 21/2013'}
{'Ley 22/2011'}
{'Orden APM/1007/2017'}
{'Ley 22/2011'}
{'Ley 22/2011'}
{'Ley 22/2011'}
{'Ley 22/2011'}
{'Ley 22/2011'}
{'Ley 22/2011'}
{'Ley 22/2011'}
{'Real Decreto 100/2011'}
{'Ley 34/2007'}
{'Real Decreto 105/2008'}
{'Real Decreto 1432/2008'}
{'Real Decreto 1528/2012'}
{'Ley 31/1995'}
{'Real Decreto 1528/2012'}
{'Real Decreto 1432/2008'}
{'Real Decreto 952/1997', 'Ley 20/1986'}
{'Ley 31/1995'}
{'Ley 31/1995'}
{'Ley 21/2013'}
{'Ley 9/2018'}
{'Ley 21/2013'}
{'Ley 21/2013'}
{'Real Decreto 1432/2008'}
{'Ley 9/2003'}
{'Ley 9/

In [259]:
for i, row_to_compare in DOCM_sumarios.iterrows():
    for j, row_comparing in ASECORP_BBDD_BOE.iterrows():
        if set(row_to_compare['Tags']) & set(row_comparing['Tags']):
            DOCM_sumarios['Match_ASECORP_BBDD'][i].append (ASECORP_BBDD_BOE['Codigo'][j])
            print(str(set(row_to_compare['Tags']) & set(row_comparing['Tags'])) + ' ' + str(row_comparing['Codigo']))

{'Ley 5/2013'} LEY 5/2013 ESP
{'Ley 21/2013'} LEY 21/2013 ESP
{'Ley 31/1995'} LEY 31/1995 ESP
{'Ley 42/2007'} LEY 33/2015 ESP
{'Ley 42/2007'} LEY 42/2007 ESP
{'Ley 9/2017'} LEY 9/2017 ESP
{'Ley 21/2013'} LEY 9/2018 ESP
{'Ley 42/2007'} RD 1274/2011 ESP
{'Ley 31/1995'} RD 171/2004 ESP
{'Real Decreto Legislativo 1/2013'} RDG 1/2013 ESP
{'Ley 31/1995'} RES 29/5/2017 ESP
{'Ley 31/1995'} RES 5/11/2010 ESP
{'Ley 21/2013'} RES 8/3/2016 ESP
{'Ley 21/2013'} STC 13/2015 ESP 
{'Ley 21/2013'} STC 53/2017 ESP
{'Resolución de 27 de julio de 2017'} RES 27/7/2017 ESP
{'Ley 36/2011'} LEY 36/2011 ESP
{'Ley 36/2011'} LEY 36/2011 ESP
{'Ley 26/2015'} LEY 26/2015 ESP
{'Ley 20/1986'} LEY 20/1986 ESP
{'Ley 21/2013'} LEY 21/2013 ESP
{'Ley 22/2011'} LEY 22/2011 ESP
{'Ley 31/1995'} LEY 31/1995 ESP
{'Ley 34/2007'} LEY 34/2007 ESP
{'Ley 22/2011'} LEY 5/2013 ESP
{'Ley 9/2018', 'Ley 21/2013'} LEY 9/2018 ESP
{'Ley 22/2011'} ORD AAA/699/2016 ESP
{'Orden APM/1007/2017'} ORD APM/1007/2017 ESP
{'Ley 22/2011'} ORD APM/189/

In [260]:
DOCM_sumarios

,seccion,item_Title,item_urlHTML,item_urlPDF,NID,numero_diario,numero_pagina,rango,organo_emisor,Fecha_publicacion,Tags,Match_ASECORP_BBDD
3,['III.- OTRAS DISPOSICIONES Y ACTOS'],"Gabinete Jurídico. Resolución de 01/03/2021, d...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/2370,45/2021,9563,Resolución,Vicepresidencia,2021-03-08,"[Ley 5/2013, Decreto 77/2019]",[LEY 5/2013 ESP]
4,['III.- OTRAS DISPOSICIONES Y ACTOS'],"Notificaciones. Notificación de 22/02/2021, de...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/2359,45/2021,9566,Notificación,Secretaría General,2021-03-08,"[Ley 29/1998, Ley 39/2015]",[]
5,['III.- OTRAS DISPOSICIONES Y ACTOS'],Ayudas y Subvenciones. Resolución de 02/03/202...,https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/2523,45/2021,9567,Resolución,Dirección General de Empresas,2021-03-08,"[Ley 9/1999, Decreto 21/2008, Ley 21/2013, Ley...","[LEY 21/2013 ESP, LEY 31/1995 ESP, LEY 33/2015..."
8,['III.- OTRAS DISPOSICIONES Y ACTOS'],Administración Electrónica. Resolución de 25/0...,https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/2348,45/2021,9636,Resolución,Viceconsejería de Administración Local y Coord...,2021-03-08,"[Resolución de 27 de julio de 2017, Resolución...",[RES 27/7/2017 ESP]
9,['III.- OTRAS DISPOSICIONES Y ACTOS'],"Espectáculos. Resolución de 01/03/2021, de la ...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/2302,45/2021,9637,Resolución,Delegación Provincial de Hacienda y Administra...,2021-03-08,"[Decreto 73/2014, Decreto 38/2013, Decreto 60/...",[]
...,...,...,...,...,...,...,...,...,...,...,...,...
95,['V.- ANUNCIOS'],"Anuncio de 25/02/2021, del Ayuntamiento de Par...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/2141,45/2021,9741,Anuncio,Ayuntamiento de Parrillas (Toledo),2021-03-08,"[Decreto 242/04, Ley 2/98]",[]
96,['V.- ANUNCIOS'],"Anuncio de 19/02/2021, del Ayuntamiento de Tam...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/1834,45/2021,9742,Anuncio,Ayuntamiento de Tamajón (Guadalajara),2021-03-08,[Decreto 242/2004],[]
97,['V.- ANUNCIOS'],"Anuncio de 19/02/2021, del Ayuntamiento de Tam...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/1835,45/2021,9743,Anuncio,Ayuntamiento de Tamajón (Guadalajara),2021-03-08,[Decreto 242/2004],[]
98,['V.- ANUNCIOS'],"Anuncio de 23/02/2021, del Ayuntamiento de Tem...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/1980,45/2021,9744,Anuncio,Ayuntamiento de Tembleque (Toledo),2021-03-08,[Decreto 242/2004],[]


In [261]:
DOCM_sumarios['Match_ASECORP_BBDD']

3                                      [LEY 5/2013 ESP]
4                                                    []
5     [LEY 21/2013 ESP, LEY 31/1995 ESP, LEY 33/2015...
8                                   [RES 27/7/2017 ESP]
9                                                    []
                            ...                        
95                                                   []
96                                                   []
97                                                   []
98                                                   []
99                                                   []
Name: Match_ASECORP_BBDD, Length: 63, dtype: object

In [262]:
DOCM_sumarios['Tags']

3                         [Ley 5/2013, Decreto 77/2019]
4                            [Ley 29/1998, Ley 39/2015]
5     [Ley 9/1999, Decreto 21/2008, Ley 21/2013, Ley...
8     [Resolución de 27 de julio de 2017, Resolución...
9     [Decreto 73/2014, Decreto 38/2013, Decreto 60/...
                            ...                        
95                           [Decreto 242/04, Ley 2/98]
96                                   [Decreto 242/2004]
97                                   [Decreto 242/2004]
98                                   [Decreto 242/2004]
99                                                   []
Name: Tags, Length: 63, dtype: object

# Genera Fichero EXCEL de resultados

In [263]:
DOCM_sumarios_final = DOCM_sumarios

In [264]:
## Cambia orden de columnas y elimina las no necesarias  
DOCM_sumarios_final.rename(columns={'NID': 'Item_id', 'item_Title': 'Item_Title', 'item_urlPDF': 'PDF_Link'}, inplace=True)
DOCM_sumarios_final = DOCM_sumarios[['Item_id','Item_Title','PDF_Link','Fecha_publicacion','Tags','Match_ASECORP_BBDD']]

In [265]:
## Crea función que convierte lista a string en todas las columnas de DOCM_sumarios
## para evitar en presentación final los caracteres [' '] propios de las listas
def list2Str(lst):
    if type(lst) is list: # apply conversion to list columns
        return", ".join(lst)
    else:
        return lst

In [266]:
## Aplica función de conversión de listas a strings
DOCM_sumarios_final = DOCM_sumarios_final.apply(lambda x: [list2Str(i) for i in x])

In [267]:
DOCM_sumarios[0:10]

,seccion,Item_Title,item_urlHTML,PDF_Link,Item_id,numero_diario,numero_pagina,rango,organo_emisor,Fecha_publicacion,Tags,Match_ASECORP_BBDD
3,['III.- OTRAS DISPOSICIONES Y ACTOS'],"Gabinete Jurídico. Resolución de 01/03/2021, d...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/2370,45/2021,9563,Resolución,Vicepresidencia,2021-03-08,"[Ley 5/2013, Decreto 77/2019]",[LEY 5/2013 ESP]
4,['III.- OTRAS DISPOSICIONES Y ACTOS'],"Notificaciones. Notificación de 22/02/2021, de...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/2359,45/2021,9566,Notificación,Secretaría General,2021-03-08,"[Ley 29/1998, Ley 39/2015]",[]
5,['III.- OTRAS DISPOSICIONES Y ACTOS'],Ayudas y Subvenciones. Resolución de 02/03/202...,https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/2523,45/2021,9567,Resolución,Dirección General de Empresas,2021-03-08,"[Ley 9/1999, Decreto 21/2008, Ley 21/2013, Ley...","[LEY 21/2013 ESP, LEY 31/1995 ESP, LEY 33/2015..."
8,['III.- OTRAS DISPOSICIONES Y ACTOS'],Administración Electrónica. Resolución de 25/0...,https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/2348,45/2021,9636,Resolución,Viceconsejería de Administración Local y Coord...,2021-03-08,"[Resolución de 27 de julio de 2017, Resolución...",[RES 27/7/2017 ESP]
9,['III.- OTRAS DISPOSICIONES Y ACTOS'],"Espectáculos. Resolución de 01/03/2021, de la ...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/2302,45/2021,9637,Resolución,Delegación Provincial de Hacienda y Administra...,2021-03-08,"[Decreto 73/2014, Decreto 38/2013, Decreto 60/...",[]
15,['III.- OTRAS DISPOSICIONES Y ACTOS'],"Notificaciones. Notificación de 01/03/2021, de...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/2294,45/2021,9646,Notificación,"Delegación Provincial de Educación, Cultura y ...",2021-03-08,[ley 39/2015],[]
16,['III.- OTRAS DISPOSICIONES Y ACTOS'],"Notificaciones. Notificación de 01/03/2021, de...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/2332,45/2021,9647,Notificación,"Delegación Provincial de Educación, Cultura y ...",2021-03-08,[Ley 39/2015],[]
17,['III.- OTRAS DISPOSICIONES Y ACTOS'],"Notificaciones. Notificación de 01/03/2021, de...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/2355,45/2021,9648,Notificación,"Delegación Provincial de Educación, Cultura y ...",2021-03-08,[Ley 39/2015],[]
18,['III.- OTRAS DISPOSICIONES Y ACTOS'],"Notificaciones. Notificación de 01/03/2021, de...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/2356,45/2021,9649,Notificación,"Delegación Provincial de Educación, Cultura y ...",2021-03-08,[Ley 39/2015],[]
19,['III.- OTRAS DISPOSICIONES Y ACTOS'],"Urbanismo. Orden 25/2021, de 26 de febrero, de...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/2335,45/2021,9650,Orden,Consejería de Fomento,2021-03-08,[Ley 29/1998],[]


In [268]:
## Generar hyperlink a artículo BOE en CSV "=HYPERLINK("https://www.boe.es/boe/dias/2021/02/02/pdfs/BOE-A-2021-1474.pdf";"BOE-A-2021-1474")"
## https://www.boe.es/diario_boe/xml.php?id=## https://www.boe.es/boe/dias/2021/02/02/pdfs/BOE-A-2021-1474.pdf

DOCM_sumarios_final_CSV = DOCM_sumarios_final

In [269]:
DOCM_sumarios_final_CSV['Item_id'] = '=HIPERVINCULO(' + '"' + DOCM_sumarios_final_CSV['PDF_Link'] + '";' \
                                                        + '"' + DOCM_sumarios_final_CSV['Item_id'] + '")'                                                        

In [271]:
#print(DOCM_sumarios_final_CSV['Item_id'][0])

In [272]:
# Elimina columna PDF_Link
DOCM_sumarios_final_CSV = DOCM_sumarios_final_CSV[['Item_id','Item_Title','Fecha_publicacion','Tags','Match_ASECORP_BBDD']]

In [273]:
DOCM_sumarios_final_CSV

,Item_id,Item_Title,Fecha_publicacion,Tags,Match_ASECORP_BBDD
3,"=HIPERVINCULO(""https://docm.jccm.es/portaldocm...","Gabinete Jurídico. Resolución de 01/03/2021, d...",2021-03-08,"Ley 5/2013, Decreto 77/2019",LEY 5/2013 ESP
4,"=HIPERVINCULO(""https://docm.jccm.es/portaldocm...","Notificaciones. Notificación de 22/02/2021, de...",2021-03-08,"Ley 29/1998, Ley 39/2015",
5,"=HIPERVINCULO(""https://docm.jccm.es/portaldocm...",Ayudas y Subvenciones. Resolución de 02/03/202...,2021-03-08,"Ley 9/1999, Decreto 21/2008, Ley 21/2013, Ley ...","LEY 21/2013 ESP, LEY 31/1995 ESP, LEY 33/2015 ..."
8,"=HIPERVINCULO(""https://docm.jccm.es/portaldocm...",Administración Electrónica. Resolución de 25/0...,2021-03-08,"Resolución de 27 de julio de 2017, Resolución ...",RES 27/7/2017 ESP
9,"=HIPERVINCULO(""https://docm.jccm.es/portaldocm...","Espectáculos. Resolución de 01/03/2021, de la ...",2021-03-08,"Decreto 73/2014, Decreto 38/2013, Decreto 60/2...",
...,...,...,...,...,...
95,"=HIPERVINCULO(""https://docm.jccm.es/portaldocm...","Anuncio de 25/02/2021, del Ayuntamiento de Par...",2021-03-08,"Decreto 242/04, Ley 2/98",
96,"=HIPERVINCULO(""https://docm.jccm.es/portaldocm...","Anuncio de 19/02/2021, del Ayuntamiento de Tam...",2021-03-08,Decreto 242/2004,
97,"=HIPERVINCULO(""https://docm.jccm.es/portaldocm...","Anuncio de 19/02/2021, del Ayuntamiento de Tam...",2021-03-08,Decreto 242/2004,
98,"=HIPERVINCULO(""https://docm.jccm.es/portaldocm...","Anuncio de 23/02/2021, del Ayuntamiento de Tem...",2021-03-08,Decreto 242/2004,


In [274]:
DOCM_sumarios_final_CSV.to_csv("./ASECORP/Resultados_Matching_DOCM_" + today.strftime("%Y%m%d") + ".csv" ,index=False) 